In [ ]:
#
# Copyright (C) 2024 Intel Corporation.
#
# SPDX-License-Identifier: Apache-2.0
#

In [1]:
!pip install tensorflow keras keras-cv opencv-contrib-python paho-mqtt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import os
import tensorflow as tf
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image
import cv2
import paho.mqtt.publish as publish
from argparse import ArgumentParser, SUPPRESS
import time

2024-11-14 15:58:15.743097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731625095.768015 3357122 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731625095.773779 3357122 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 15:58:15.796681: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model = tf.keras.applications.ResNet50(weights='imagenet')

2024-11-14 15:58:18.897238: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
def perform_detection(img):
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x = tf.expand_dims(x, axis=0)
    predictions = model.predict(x)
    decoded_predictions = decode_predictions(predictions, top=2)[0]
    return decoded_predictions

def process_video(video_path, mqtt_broker_hostname, mqtt_broker_port, mqtt_outgoing_topic):
    cap = cv2.VideoCapture(video_path)

    t0 = time.time()
    frame_count = 0
    cumulative_fps = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, (224, 224))
        results = perform_detection(img_resized)
               
        elapsed_time = time.time() - t0
        frame_count += 1
        cumulative_fps = frame_count / elapsed_time
        print(f"Cumulative Average FPS: {cumulative_fps: .2f}")

        print(results)
        for i, (_, label, prob) in enumerate(results):
                if (prob > 0.5):
                    cv2.putText(frame, f"{label}: {prob:.2f}", (10, 30 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    detected_products_jsonMsg = f"{label}: {prob:.2f}"
                    publish_mqtt_msg(
                        detected_products_jsonMsg, mqtt_broker_hostname, mqtt_broker_port,
                        mqtt_outgoing_topic
                    )

        cv2.imshow('Tensorflow-Keras Classification', frame)
        if cv2.waitKey(1) == ord('q'):
            break
       
    cap.release()
    cv2.destroyAllWindows()

def publish_mqtt_msg(
        detected_products, mqtt_broker_hostname, mqtt_broker_port, mqtt_outgoing_topic):
    publish.single(
        mqtt_outgoing_topic, str(detected_products),  hostname=mqtt_broker_hostname, port=mqtt_broker_port
    )

In [5]:
video_path = "/home/nesubuntu207/Desktop/coca-cola-4465029-3840-15-bench.mp4"
mqtt_broker_hostname = "localhost"
mqtt_broker_port = 1883
mqtt_outgoing_topic = "tensorflow_keras_classification_results"

In [6]:
process_video(video_path, mqtt_broker_hostname, mqtt_broker_port, mqtt_outgoing_topic)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Cumulative Average FPS:  0.40
[('n03983396', 'pop_bottle', 0.78087926), ('n02823428', 'beer_bottle', 0.12830734)]


QObject::moveToThread: Current thread (0xfc83220) is not the object's thread (0xb6e7d80).
Cannot move to target thread (0xfc83220)

QObject::moveToThread: Current thread (0xfc83220) is not the object's thread (0xb6e7d80).
Cannot move to target thread (0xfc83220)

QObject::moveToThread: Current thread (0xfc83220) is not the object's thread (0xb6e7d80).
Cannot move to target thread (0xfc83220)

QObject::moveToThread: Current thread (0xfc83220) is not the object's thread (0xb6e7d80).
Cannot move to target thread (0xfc83220)

QObject::moveToThread: Current thread (0xfc83220) is not the object's thread (0xb6e7d80).
Cannot move to target thread (0xfc83220)

QObject::moveToThread: Current thread (0xfc83220) is not the object's thread (0xb6e7d80).
Cannot move to target thread (0xfc83220)

QObject::moveToThread: Current thread (0xfc83220) is not the object's thread (0xb6e7d80).
Cannot move to target thread (0xfc83220)

QObject::moveToThread: Current thread (0xfc83220) is not the object's thread

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
Cumulative Average FPS:  0.54
[('n03983396', 'pop_bottle', 0.7860762), ('n02823428', 'beer_bottle', 0.12165092)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Cumulative Average FPS:  0.71
[('n03983396', 'pop_bottle', 0.78945786), ('n02823428', 'beer_bottle', 0.120234124)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Cumulative Average FPS:  0.89
[('n03983396', 'pop_bottle', 0.78835076), ('n02823428', 'beer_bottle', 0.12162838)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Cumulative Average FPS:  1.06
[('n03983396', 'pop_bottle', 0.7835515), ('n02823428', 'beer_bottle', 0.12469276)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Cumulative Average FPS:  1.20
[('n03983396', 'pop_bottle', 0.7867644), ('n02823428', 'beer_bottle', 0.1224281)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Cumulative Average FPS:  1.34
[('n03983396', 'pop_bottle', 0.7829847), ('n02823428', 'beer_bottle', 0.12609741)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Cumulative Average FPS:  1.46
[('n03983396', 'p